## Dependencies

In [123]:
# Dependencies
import pandas as pd
import numpy as np
from datetime import date
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, r2_score
import xgboost as xgb
import matplotlib.pyplot as plt
import json

In [124]:
# Path to the Excel file
excel_file = f'data/Hellman_Top20.xlsx'

# List of Retailers
sheet_names = ['Walmart', 'LCL', "SDM"]  

# Initialize an empty DataFrame for merging
merged_df = pd.DataFrame()

# Read each sheet into a DataFrame and merge them
for retailer in sheet_names:
    df = pd.read_excel(excel_file, sheet_name=retailer, index_col='Date')
    df.index = pd.to_datetime(df.index)
    df.columns = [f'{retailer} | {col}' for col in df.columns]
    # Merge with the accumulated DataFrame
    if merged_df.empty:
        merged_df = df
    else:
        merged_df = pd.merge(merged_df, df, left_index=True, right_index=True, how='left')

df = merged_df.copy()

#set up the target var
sales = [sku for sku in df.columns if 'Pricing' not in sku]
price_features = [sku for sku in df.columns if 'Pricing' in sku]
temporal_features = [
    'quarter',
    'month',
    'year',
    'weekofyear',
    'weekofmonth'
]

# Spend
spend_df = pd.read_excel(excel_file, sheet_name="Spend", index_col='Date')
spend_df.index = pd.to_datetime(spend_df.index)
spend_list = spend_df.columns

# Add media features to the dataframe
media_df = pd.read_excel(excel_file, sheet_name='Media', index_col='Date')
media_df.index = pd.to_datetime(df.index)
### filter the media thats less than 10k spend
droplist=[]
for media in media_df.columns:
    media_spend = media.replace("Impressions", "Spend")
    if media_spend in spend_list and spend_df[media_spend].sum()<10000:
        droplist.append(media)
media_df.drop(columns=droplist, inplace=True)
######
cat_features_withoutlag = media_df.columns
df = pd.merge(df, media_df, left_index=True, right_index=True, how='left')

cat_features_withoutlag

Index(['Social | Hellman | Dressings | Meta - Impressions',
       'Social | Hellman | Dressings | Pinterest - Impressions',
       'Social | Hellman | Dressings | TikTok - Impressions',
       'Programmatic | Hellman | Deodorant | Amazon | Video - Impressions',
       'Programmatic | Hellman | Deodorant | DV360 | Video - Impressions',
       'Programmatic | Hellman | Nutrition | Amazon | Video - Impressions',
       'Programmatic | Hellman | Nutrition | DV360 | Display - Impressions',
       'Programmatic | Hellman | Nutrition | DV360 | Video - Impressions',
       'Programmatic | Hellman | Nutrition | TTD | Audio - Impressions',
       'Programmatic | Hellman | Nutrition | TTD | Display - Impressions',
       'Programmatic | Hellman | Nutrition | TTD | Video - Impressions',
       'Instacart | Hellman | Dressing - Impressions',
       'Amazon Search | Hellman | Food - Impressions',
       'Google Ads | Hellman | Nutrition - Impressions',
       'TV | Hellmanns - Nutrition - English -

In [125]:
len(droplist)/(len(droplist)+len(media_df.columns))

0.13636363636363635

In [126]:
# Access the environment variable, enter the category here
CATEGORY = 'Hellman'
portion = 0.99
params = {
            "base_score": 0.5,
            "booster": "gbtree",
            "n_estimators": 3000,
            "early_stopping_rounds": 150,
            "objective": "reg:absoluteerror",
            "max_depth": 3,
            "learning_rate": 0.01,
            "colsample_bytree": 0.9,
            "reg_alpha": 0.1,
            "reg_lambda": 10
        }
cat_list = cat_features_withoutlag
print(CATEGORY)

Hellman


## Forecaster

#### Load the Data

#### Add temporal features and lagged features

In [127]:
# Extract temporal features
df['quarter'] = df.index.quarter
df['month'] = df.index.month
df['year'] = df.index.year
df['weekofyear'] = df.index.isocalendar().week
df['weekofmonth'] = df.index.to_series().apply(lambda x: (x.day - 1) // 7 + 1)
#create lag features
lag = 2
cat_features_withlag = []
for f in cat_features_withoutlag:
    cat_features_withlag.append(f)
    for l in range(1, lag+1):
        df[f'{f}_lag_{l}'] = df[f].shift(l)
        cat_features_withlag.append(f'{f}_lag_{l}')

In [128]:
# Dimension reduction using the selected media features - "cat_features_withlag"
df = df[temporal_features+sales+price_features+cat_features_withlag]
df.head(3)

,quarter,month,year,weekofyear,weekofmonth,Walmart | 068400662600,Walmart | 068400616207,Walmart | 068400662907,Walmart | 068400662709,Walmart | 068400142157,...,TV | Hellmanns - Nutrition - French - 15 - Impressions_lag_2,DirectIO | Hellman | Nutrition | Samsung Ads - Impressions,DirectIO | Hellman | Nutrition | Samsung Ads - Impressions_lag_1,DirectIO | Hellman | Nutrition | Samsung Ads - Impressions_lag_2,DirectIO | Hellman | Nutrition | Stingray - Impressions,DirectIO | Hellman | Nutrition | Stingray - Impressions_lag_1,DirectIO | Hellman | Nutrition | Stingray - Impressions_lag_2,DirectIO | Hellman | Nutrition | Zefr - Impressions,DirectIO | Hellman | Nutrition | Zefr - Impressions_lag_1,DirectIO | Hellman | Nutrition | Zefr - Impressions_lag_2
Date,,,,,,,,,,,,,,,,,,,,,
2023-01-08,1,1,2023,1,2,184182.96,119578.81,60694.38,62066.28,59025.40,...,NaN,0,NaN,NaN,0.0,NaN,NaN,0,NaN,NaN
2023-01-15,1,1,2023,2,3,153627.07,101623.65,69174.49,52504.76,53881.80,...,NaN,0,0.0,NaN,0.0,0.0,NaN,0,0.0,NaN
2023-01-22,1,1,2023,3,4,150639.48,97005.86,65364.36,49558.11,49828.73,...,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0


In [129]:
# the total volumn of sales for the category you selected
df[sales].sum().sum()

192523309.57000002

## train

In [130]:
# List to store individual models
models = []
results = []

# Training loop for each output
for i in range(len(sales)):
    # train test split
    FEATURES = temporal_features + [price_features[i]] + cat_features_withlag
    TARGET = sales[i]
    df_copy = df.dropna(subset=[TARGET]+FEATURES) #subset=[TARGET, price_features[i]]
    print(i+1)
    # print(df.shape[0] - df_copy.shape[0]) #how many data rows are dropped
   
    # Drop rows with NaN values to get the valid sales data
    valid_sales_df = df[TARGET].dropna()
    split_index = int(len(valid_sales_df) * portion)# Determine the split index based on 75% for training and 25% for testing
    split_date = valid_sales_df.index[split_index]

    train = df_copy.loc[df_copy.index < split_date]
    test = df_copy.loc[df_copy.index >= split_date]

    X_train = train[FEATURES]
    y_train = train[TARGET]
    X_test = test[FEATURES]
    y_test = test[TARGET]
    
    monotonic_constraints = tuple([1 if feature in cat_features_withlag else 0 for feature in FEATURES])

    reg = xgb.XGBRegressor(**params,
                       monotone_constraints=monotonic_constraints
                       )
    
    reg.fit(
        X_train, y_train,
        eval_set=[(X_test, y_test)],
        verbose=0
    )
    
    # Save the model to local
    # reg.save_model(f'../DoveModel/{CATEGORY}/model_{i+1}.json')

    models.append([reg, FEATURES])

    # forecasting
    def mape(y_true, y_pred):
        y_true, y_pred = np.array(y_true), np.array(y_pred)
        return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    
    y_pred = reg.predict(X_test)
    y_true = y_test
    results.append(mape(y_true, y_pred))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53


In [131]:
results

[15.677658478663362,
 18.904307555559573,
 16.924863858528262,
 3.307051491468348,
 0.7965662921519605,
 0.023865100312370983,
 4.853692024175637,
 0.05464291289671604,
 0.37310375454245326,
 0.01525912039183725,
 30.157965474172133,
 1.427466563237193,
 2.1853263298844223,
 26.691056160798748,
 91.29745460153684,
 6.809446346034659,
 12.340099189433454,
 0.5535159161230189,
 10.3160768212097,
 17.081332865868536,
 14.89882459705187,
 13.362195471170526,
 9.181810017213571,
 0.008739516578549688,
 12.022205517500652,
 25.828778126289066,
 8.051550972446792,
 13.233303974216717,
 0.0066054737112845084,
 0.038365627176281066,
 11.234078651758495,
 19.889518840748135,
 0.008507636856008808,
 16.959380253950627,
 6.224167334619885,
 0.016153131856981133,
 0.0075158689240242054,
 3.4238075097056835,
 0.032945361907703266,
 4.125742688383174,
 0.015010219956448472,
 13.75023813640345,
 0.4716277987586696,
 6.260636935345267,
 1.2198336984558529,
 4.211775461832682,
 3.534573946143047,
 2.251

In [132]:
# Predict on test set
print(np.mean(results))
print(len(results))

13.790796209022696
53


## Inference

## Yearly ROI

In [135]:
yearly_df = pd.DataFrame(columns=["Year", CATEGORY, "Revenue", "Spend"])
for year, group in df.groupby(pd.Grouper(freq="YE")):
    year= year.year
    TIME_START = group.index.min()
    TIME_END = group.index.max()
    
    # merge spending data to the df
    merged_df = df.copy()
    min_df = merged_df.min()
    merged_df = merged_df.loc[TIME_START:TIME_END+pd.DateOffset(14)] # select defined time range

    for media in cat_features_withoutlag:
        X_ori = merged_df.copy()
        X_zero = merged_df.copy()
        # #get min
        minimum_value = 0 #min_df[media]
        # print(minimum_value)
        X_zero.loc[TIME_START:TIME_END, f"{media}"] = minimum_value
        X_zero.loc[TIME_START+pd.DateOffset(7):TIME_END+pd.DateOffset(7), f"{media}_lag_1"] = minimum_value
        X_zero.loc[TIME_START+pd.DateOffset(14):TIME_END+pd.DateOffset(14), f"{media}_lag_2"] = minimum_value
        revenue_list = []
        for i, model in enumerate(models):
            model, features = model
            X_ori_copy = X_ori.dropna(subset=features)
            X_zero_copy = X_zero.dropna(subset=features)
            
            original_pred = model.predict(X_ori_copy[features])
            zero_pred = model.predict(X_zero_copy[features])
            revenue_list.append(sum(np.maximum(original_pred-zero_pred, 0))) # return on sku level
        revenue = sum(revenue_list)
        spend_col = media.replace("Impressions", "Spend")
        if spend_col not in spend_list:
            yearly_df.loc[len(yearly_df)] = [year, media, revenue, np.nan]
        else:
            media_spend = spend_df.loc[TIME_START:TIME_END, spend_col].sum()
            yearly_df.loc[len(yearly_df)] = [year, media, revenue, media_spend]

# Add a new column called ROI
yearly_df['ROI'] = yearly_df.apply(lambda row: round(row['Revenue'] / row['Spend'], 2) if row['Spend'] not in [None, 0] else np.nan, axis=1)
# yearly_df = yearly_df.sort_values(by='Spend', ascending=False)

# Insert the new columns and drop the old one
split_cols = yearly_df[CATEGORY].str.split(r' \| ', n=1, expand=True)
yearly_df.insert(1, 'Category', split_cols[0])
yearly_df[CATEGORY] = split_cols[1]
yearly_df

,Year,Category,Hellman,Revenue,Spend,ROI
0,2023,Social,Hellman | Dressings | Meta - Impressions,1.309536e+06,8.447682e+05,1.55
1,2023,Social,Hellman | Dressings | Pinterest - Impressions,1.689832e+05,3.164998e+05,0.53
2,2023,Social,Hellman | Dressings | TikTok - Impressions,2.199350e+05,1.262318e+05,1.74
3,2023,Programmatic,Hellman | Deodorant | Amazon | Video - Impress...,0.000000e+00,0.000000e+00,NaN
4,2023,Programmatic,Hellman | Deodorant | DV360 | Video - Impressions,0.000000e+00,0.000000e+00,NaN
5,2023,Programmatic,Hellman | Nutrition | Amazon | Video - Impress...,4.695897e+04,7.510579e+04,0.63
6,2023,Programmatic,Hellman | Nutrition | DV360 | Display - Impres...,3.692509e+01,3.719977e+04,0.00
7,2023,Programmatic,Hellman | Nutrition | DV360 | Video - Impressions,9.368227e+04,4.870404e+05,0.19
8,2023,Programmatic,Hellman | Nutrition | TTD | Audio - Impressions,1.454851e+04,2.974709e+04,0.49
9,2023,Programmatic,Hellman | Nutrition | TTD | Display - Impressions,3.122026e+04,2.041485e+05,0.15


In [136]:
yearly_df.to_excel("hellman_xgb_output.xlsx", index=False)